# Data Validation
Contents:
1. Load multi-year datasets & define QC scope
2. Define meta fields, indicator groups, and analysis domains (PHChemical / PHPhysical)
3. Filter rules (surface/subsurface, primary sample, land-use subsets)
4. Duplicate handling (key definition, removal logic, audit tables)
5. Non-detect handling (<RL parsing, RL map, substitution rule)
6. Missing value handling policy (distinguish “missing” vs “non-detect”)
7. Data type standardisation & unit sanity checks (numeric coercion, invalid tokens)
8. Outlier detection & removal (Z-score threshold / Grubbs-style rule)
9. QC audit outputs (per-year QC log, RL summary, outlier summary, completeness metrics)
10. Export QC-processed datasets (year-wise) + reproducibility metadata

**Imports & Settings**

In [1]:
import pandas as pd
import numpy as np
import os

PROJECT_ROOT = r'D:/实习/工作/1_遥感检测土壤污染/EAD to GSS'

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

**Inputs & Load multi-year datasets**

In [2]:
path_list = [
    r'Soil quality data/Soil quality data 2020.xlsx',
    r'Soil quality data/Soil quality data 2021.xlsx',
    r'Soil quality data/Soil quality data 2022.xlsx',
    r'Soil quality data/Soil quality data 2023.xlsx',
    r'Soil quality data/Soil quality data 2024.xlsx',
    r'Soil quality data/Soil quality data 2025.xlsx',
]

data_dict = {}
for path in path_list:
    temp = pd.read_excel(os.path.join(PROJECT_ROOT, path), sheet_name=None)
    temp_name = os.path.basename(path).replace(".xlsx", "")
    data_dict[temp_name] = temp

YEARS = list(data_dict.keys())
pd.DataFrame({"Year": YEARS})

,Year
0,Soil quality data 2020
1,Soil quality data 2021
2,Soil quality data 2022
3,Soil quality data 2023
4,Soil quality data 2024
5,Soil quality data 2025


**Config, GROUPS, META_COLS**

In [3]:
GROUPS = {
    "Inorganics (including heavy metals)": [
        "Aluminium", "Antimony", "Arsenic", "Barium", "Beryllium", "Boron_aqua",
        "Cadmium", "Calcium", "Chromium_Total", "Chromium_VI", "Cobalt", "Copper",
        "Iron_aqua", "Lead_aqua", "Lithium", "Magnesium", "Manganese", "InorganicMercury",
        "Molybdenum", "Nickel_aqua", "Phosphorus", "Potassium", "Selenium_aqua", "Silver",
        "Sodium", "Strontium", "Thallium", "Tin", "Titanium", "Uranium", "Vanadium", "Zinc",
    ],
    "Petroleum Hydrocarbons": ["EPH_C10_C40", "VPH_C5_C10"],
    "Polycyclic Aromatic Hydrocarbons (PAHs)": [
        "Acenaphthene", "Acenaphthylene", "Anthracene", "Benzo_a_anthracene", "Benzo_a_pyrene",
        "Benzo_b_fluoranthene", "Benzo_g_h_i_perylene", "Benzo_k_fluoranthene", "Chrysene",
        "Dibenz_a_h_anthracene", "Fluoranthene", "Fluorene", "Indeno_1_2_3_c_d_pyrene",
        "Naphthalene", "Phenanthrene", "Pyrene", "Total_PAHs_BaP_equivalents",
    ],
    "Polychlorinated Biphenyls (PCBs)": [
        "Heptachlorobiphenyl_2_3_3_4_4_5_5_PCB189", "Hexachlorobiphenyl_2_3_4_4_5_5_PCB167",
        "Hexachlorobiphenyl_2_3_3_4_4_5_PCB_156", "Hexachlorobiphenyl_2_3_3_4_4_5_PCB157",
        "Hexachlorobiphenyl_3_3_4_4_5_5_PCB_169", "Pentachlorobiphenyl_2_3_3_4_4_PCB105",
        "Pentachlorobiphenyl_2_3_4_4_5_PCB_114", "Pentachlorobiphenyl_2_3_4_4_5_PCB_118",
        "Pentachlorobiphenyl_2_3_4_4_5_PCB123", "Pentachlorobiphenyl_3_3_4_4_5_PCB_126",
        "Tetrachlorobiphenyl_3_3_4_4_PCB77", "Tetrachlorobiphenyl_3_4_4_5_PCB_81",
    ],
    "Organochlorine Pesticides (OCPs)": [
        "Aldrin", "BHC_alpha", "BHC_beta", "BHC_delta", "BHC_gamma_Lindane", "Chlordane",
        "DDD", "DDE", "DDT", "Dieldrin", "Endosulfan_alpha", "Endosulfan_beta",
        "Endosulfan_sulphate", "Endrin", "Endrin_aldehyde", "Heptachlor",
        "Heptachlor_epoxide", "Methoxychlor",
    ],
    "Organophosphorus Pesticides (OPs)": [
        "Bromophos_ethyl", "Bromophos_methyl", "Cadusafos", "Chlorpyrifos", "Dichlorvos",
        "Disulfoton", "Ethoprophos", "Famphur", "Fenchlorphos", "Guthion",
        "o_o_o_triethylphosphorothionate", "Parathion", "Sulfotep", "Tokuthion",
    ],
    "Other Pesticides (Non-OP / Non-OCP)": [
        "Acetamiprid", "Atrazine_desethyl", "Azoxystrobin", "Bifenthrin", "Carboxin",
        "Chlorfenapyr", "Clothianidin", "Cyfluthrin", "Cypermethrin_I_Zeta", "Cypermethrin_II_Teta",
        "Cypermethrin_III_Beta", "Cypermethrin_IV_Alpha", "Cyprodinil", "Cyromazine",
        "Deltamethrin", "Difenoconazole", "Dimethomorph", "Dimetilan", "Diniconazole",
        "Epoxiconazole", "Fenarimol", "Fenhexamid", "Fenpropathrin", "Fenpropimorph",
        "Fludioxonil", "Fluopyram", "Fluquinconazole", "Flutolanil", "Flutriafol",
        "Hexaconazole", "Hexythiazox", "Imazalil", "Imidacloprid", "Lufenuron",
        "Metalaxyl", "Methoxyfenozid", "Metrafenone", "Oxadixyl", "Paraquat", "Pebulate",
        "Permethrin", "Prochloraz", "Procymidone", "Propiconazole", "Prothioconazole",
        "Pyridaben", "Pyriproxyfen", "Quinoxyfen", "Resmethrin", "Tebuconazole", "Tefluthrin",
        "Thiabendazole", "Thiacloprid", "Thiamethoxam", "Thiophanate_Methyl", "Tolclofos_methyl",
        "Tolfenpyrad", "Triadimenol", "Triflumuron",
    ],
    "Physico-chemical & Soil Properties": [
        "Nitrogen Kjeldhal, dry mass", "SAR", "sar", "satB_mgl", "satca", "satec",
        "satk", "satmg", "satna", "satph", "TiCl4", "TotalKjeldahlNitrogen_TKN", "TotalOrganicCarbon",
    ],
}

META_COLS = [
    "SurveyID", "SiteUID", "SiteObsUID", "PedonUID", "PHUID",
    "PHSampleUID", "PHSubSampleUID", "PHChemicalUID",
    "Longitude", "Latitude", "obsdate", "Location",
]

# flatten indicators list
elements = []
for g in GROUPS.values():
    elements.extend(g)

# switches
Z_THRESHOLD = 3.8
FILL_NA_AS_NONDETECT = False

OUT_DIR = os.path.join(PROJECT_ROOT, r'Result_hmq/Soil quality data QC')
os.makedirs(OUT_DIR, exist_ok=True)

pd.DataFrame({
    "Z_THRESHOLD": [Z_THRESHOLD],
    "FILL_NA_AS_NONDETECT": [FILL_NA_AS_NONDETECT],
    "OUT_DIR": [OUT_DIR],
})


,Z_THRESHOLD,FILL_NA_AS_NONDETECT,OUT_DIR
0,3.8,False,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Soil...


**QC function**

In [4]:
def qc_phchemical_one_year(
    data: dict,
    year_label: str,
    elements: list,
    meta_cols_master: list,
    z_threshold: float = 3.8,
    fill_na_as_nondetect: bool = False,
    phuid_surface: int = 1,
    primary_sample_uid: int = 1,
):
    """
    QC for one year (PHChemical):
    1) filter surface + primary samples
    2) df_raw = meta + valid_elements
    3) handle non-detect (<RL -> 0.5*RL) + numeric coercion + optional NA fill
    4) outlier removal via z-score |z|>threshold -> NaN
    Returns:
        df_filtered, df_raw, df_processed, qc_report_nd, qc_report_outlier, qc_run_summary
    """
    phc0 = data["PHChemical"].copy()

    # --- filters ---
    df_filtered = phc0.loc[phc0["PHUID"] == phuid_surface].copy()
    df_filtered = df_filtered.loc[df_filtered["PHSampleUID"] == primary_sample_uid].copy()

    # --- valid cols ---
    valid_elements = [e for e in elements if e in df_filtered.columns]
    meta_cols = [c for c in meta_cols_master if c in df_filtered.columns]

    df_raw = df_filtered.loc[:, meta_cols + valid_elements].copy()

    # --- non-detect handling ---
    df = df_raw.copy()
    rl_map = {}
    qc_counts = []

    for col in valid_elements:
        s = df[col]
        s_str = s.astype(str).str.strip()
        mask_lt = s_str.str.startswith("<", na=False)

        rl_series = pd.to_numeric(s_str[mask_lt].str[1:], errors="coerce").dropna()
        rl = None

        if not rl_series.empty:
            rl = float(rl_series.max())  # conservative
            rl_map[col] = rl
            df.loc[mask_lt, col] = 0.5 * rl

        # numeric coercion
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # optional NA fill
        na_before = int(df[col].isna().sum())
        if fill_na_as_nondetect and (rl is not None):
            df[col] = df[col].fillna(0.5 * rl)

        qc_counts.append({
            "year": year_label[-4:],
            "element": col,
            "reporting_limit_used": rl_map.get(col, np.nan),
            "n_total": len(df[col]),
            "n_non_detect_<RL": int(mask_lt.sum()),
            "n_na_after_numeric": int(df[col].isna().sum()),
            "n_na_before_fill": na_before,
            "na_filled_as_non_detect": bool(fill_na_as_nondetect and (rl is not None)),
        })

    qc_report_nd = pd.DataFrame(qc_counts)

    # --- outlier removal ---
    outlier_stats = []
    for col in valid_elements:
        x = df[col]
        mu = x.mean(skipna=True)
        sd = x.std(skipna=True)

        if (sd is None) or np.isnan(sd) or sd == 0:
            outlier_stats.append({
                "year": year_label[-4:],
                "element": col,
                "mean": mu,
                "sd": sd,
                "n_outliers_removed": 0,
                "note": "skip (sd=0 or insufficient data)",
            })
            continue

        z = (x - mu) / sd
        mask_out = z.abs() > z_threshold
        n_out = int(mask_out.sum(skipna=True))

        df.loc[mask_out, col] = np.nan

        outlier_stats.append({
            "year": year_label[-4:],
            "element": col,
            "mean": float(mu),
            "sd": float(sd),
            "n_outliers_removed": n_out,
            "note": "",
        })

    qc_report_outlier = pd.DataFrame(outlier_stats)

    # --- run summary ---
    qc_run_summary = pd.DataFrame([{
        "year": year_label[-4:],
        "n_records_before_filter": len(phc0),
        "n_records_after_filter": len(df_filtered),
        "n_sites_after_filter": df_filtered["SiteUID"].nunique() if "SiteUID" in df_filtered.columns else np.nan,
        "n_valid_elements": len(valid_elements),
        "z_threshold": z_threshold,
        "fill_na_as_nondetect": fill_na_as_nondetect,
    }])

    return df_filtered, df_raw, df, qc_report_nd, qc_report_outlier, qc_run_summary

**Run QC for ALL years**

In [5]:
all_qc_summary = []
all_qc_nd = []
all_qc_outlier = []
processed_by_year = {}   # store processed df for later export / analysis

for year_label, data in data_dict.items():
    df_filtered, df_raw, df_processed, qc_nd, qc_out, qc_sum = qc_phchemical_one_year(
        data=data,
        year_label=year_label,
        elements=elements,
        meta_cols_master=META_COLS,
        z_threshold=Z_THRESHOLD,
        fill_na_as_nondetect=FILL_NA_AS_NONDETECT,
        phuid_surface=1,
        primary_sample_uid=1,
    )

    processed_by_year[year_label] = {
        "filtered": df_filtered,
        "raw": df_raw,
        "processed": df_processed,
        "qc_nd": qc_nd,
        "qc_out": qc_out,
        "qc_sum": qc_sum,
    }

    all_qc_summary.append(qc_sum)
    all_qc_nd.append(qc_nd)
    all_qc_outlier.append(qc_out)

df_qc_summary = pd.concat(all_qc_summary, ignore_index=True)
df_qc_summary

,year,n_records_before_filter,n_records_after_filter,n_sites_after_filter,n_valid_elements,z_threshold,fill_na_as_nondetect
0,2020,619,295,295,37,3.8,False
1,2021,768,365,365,50,3.8,False
2,2022,1174,560,560,50,3.8,False
3,2023,1468,700,700,73,3.8,False
4,2024,1310,625,625,107,3.8,False
5,2025,743,709,709,36,3.8,False


**QC audit tables**

In [6]:
df_qc_nd = pd.concat(all_qc_nd, ignore_index=True)

df_qc_nd.sort_values(
    ["n_non_detect_<RL", "n_na_after_numeric"],
    ascending=False
).head(30)

,year,element,reporting_limit_used,n_total,n_non_detect_<RL,n_na_after_numeric,n_na_before_fill,na_filled_as_non_detect
339,2025,Silver,10.00,709,709,0,0,False
342,2025,Thallium,0.25,709,708,0,0,False
343,2025,Tin,1.00,709,660,0,0,False
233,2024,Silver,10.00,625,625,0,0,False
236,2024,Thallium,0.25,625,625,0,0,False
237,2024,Tin,1.00,625,583,0,0,False
239,2024,Uranium,1.60,625,560,0,0,False
345,2025,Uranium,1.60,709,552,0,0,False
333,2025,InorganicMercury,0.01,709,541,0,0,False
100,2022,InorganicMercury,0.01,560,467,0,0,False


In [7]:
df_qc_outlier = pd.concat(all_qc_outlier, ignore_index=True)

df_qc_outlier.sort_values(
    "n_outliers_removed",
    ascending=False
).head(30)

,year,element,mean,sd,n_outliers_removed,note
309,2024,satec,22.058507,40.126665,13,
343,2025,Tin,0.588287,0.401636,13,
340,2025,Sodium,4451.094602,11354.547267,12,
215,2024,Boron_aqua,18.264754,13.756369,10,
152,2023,Molybdenum,0.727052,1.003580,10,
318,2025,Antimony,0.173510,0.202404,10,
333,2025,InorganicMercury,0.010691,0.019313,10,
345,2025,Uranium,0.938624,0.402065,10,
234,2024,Sodium,2465.976875,5624.890263,9,
221,2024,Copper,7.416605,7.146591,9,


**Step-by-step inspection for one selected year**

In [8]:
YEAR = 2025
year_label = f"Soil quality data {YEAR}"
pack = processed_by_year[year_label]

df_filtered = pack["filtered"]
df_raw = pack["raw"]
df_processed = pack["processed"]
qc_nd = pack["qc_nd"]
qc_out = pack["qc_out"]
qc_sum = pack["qc_sum"]

qc_sum

,year,n_records_before_filter,n_records_after_filter,n_sites_after_filter,n_valid_elements,z_threshold,fill_na_as_nondetect
0,2025,743,709,709,36,3.8,False


**Export: output all years**

In [9]:
for year_label, pack in processed_by_year.items():
    y = year_label[-4:]
    out_path = os.path.join(OUT_DIR, f"Soil quality data QC {y}.xlsx")

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        pack["processed"].to_excel(writer, sheet_name="Processed_Data", index=False)
        pack["qc_nd"].to_excel(writer, sheet_name="QC_NonDetect_Report", index=False)
        pack["qc_out"].to_excel(writer, sheet_name="QC_Outlier_Report", index=False)
        pack["qc_sum"].to_excel(writer, sheet_name="Run_Summary", index=False)

pd.DataFrame({
    "Exported_years": [", ".join([y[-4:] for y in processed_by_year.keys()])],
    "Output_dir": [OUT_DIR],
})

,Exported_years,Output_dir
0,"2020, 2021, 2022, 2023, 2024, 2025",D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Soil...
